In [2]:
%load_ext dotenv
%dotenv
import openai
import re
import requests
import sys
from num2words import num2words
import os
import pandas as pd
import numpy as np
from openai.embeddings_utils import get_embedding, cosine_similarity
from transformers import GPT2TokenizerFast

API_KEY = os.environ.get("AZURE_OPENAI_API_KEY")
RESOURCE_ENDPOINT = os.environ.get("AZURE_OPENAI_ENDPOINT")

openai.api_type = "azure"
openai.api_key = API_KEY
openai.api_base = RESOURCE_ENDPOINT
openai.api_version = "2022-12-01"
url = openai.api_base + "/openai/deployments?api-version=2022-12-01"

r = requests.get(url, headers={"api-key": API_KEY})

print(r.text)

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


{
  "data": [
    {
      "scale_settings": {
        "scale_type": "standard"
      },
      "model": "text-ada-001",
      "owner": "organization-owner",
      "id": "Ada1",
      "status": "succeeded",
      "created_at": 1676559783,
      "updated_at": 1676559783,
      "object": "deployment"
    },
    {
      "scale_settings": {
        "scale_type": "standard"
      },
      "model": "text-embedding-ada-002",
      "owner": "organization-owner",
      "id": "embedding-model",
      "status": "succeeded",
      "created_at": 1676563276,
      "updated_at": 1676563276,
      "object": "deployment"
    },
    {
      "scale_settings": {
        "scale_type": "standard"
      },
      "model": "text-davinci-003",
      "owner": "organization-owner",
      "id": "davinchi-003",
      "status": "succeeded",
      "created_at": 1676563605,
      "updated_at": 1676563605,
      "object": "deployment"
    }
  ],
  "object": "list"
}


In [13]:
# Taken from https://azure.microsoft.com/en-us/free/free-account-faq/
df_faq = pd.read_csv("data/azure-free-account-faq.csv") 
df_faq

,question,answer
0,How do I get popular services free for 12 months?,"When you create your Azure free account, you s..."
1,How do I get the 55+ other services that are a...,You start getting monthly free amounts of more...
2,What can I do with my $200 credit?,You can apply your credit to try services that...
3,Who is eligible for the Azure free account?,The Azure free account is available to all new...
4,Do I pay anything to start with the Azure free...,No. Starting is free. You won't be charged any...
5,How do I ensure that I won’t be charged?,We’ll never charge you unless you decide to mo...
6,Do I have to pay something after 30 days or af...,At the end of your first 30 days or after you ...
7,Can I skip the 30 days and just start with pay...,Yes. You can sign up directly for pay as you g...
8,What happens once I use my $200 free credit or...,We’ll notify you so you can decide if you want...
9,What does it mean to upgrade my account to pay...,"When you sign up for an Azure free account, yo..."


In [14]:
# s is input text
def normalize_text(s, sep_token = " \n "):
    print(s)
    s = re.sub(r'\s+',  ' ', s).strip()
    s = re.sub(r". ,","",s)
    # remove all instances of multiple spaces
    s = s.replace("..",".")
    s = s.replace(". .",".")
    s = s.replace("\n", "")
    s = s.strip()
    
    return s

df_faq['answer'] = df_faq["answer"].apply(lambda x : normalize_text(x))

When you create your Azure free account, you start getting monthly free amounts of certain types of services. If you move to pay-as-you-go pricing within 30 days or after you’ve used your credit, you’ll continue to receive monthly free amounts of popular services until 12 months after you created your account. After that, if you’re using any of these services, they’ll continue to run and you’ll be billed for them at pay-as-you-go rates.
You start getting monthly free amounts of more than 55 other services when you create your Azure free account. If you move to pay-as-you-go pricing after 30 days or after you use your credit, you’ll continue to receive monthly free amounts of these services. If you exceed your monthly free amounts for any of these services, you’ll be billed for them at pay-as-you-go rates.
You can apply your credit to try services that aren't in the free list, or to use more than your free amounts of any services.
The Azure free account is available to all new Azure cus

In [63]:
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
df_faq['n_tokens'] = df_faq["answer"].apply(lambda x: len(tokenizer.encode(x)))

# remove any answers that are too long for the token limit (~2000 tokens).
df_faq = df_faq[df_faq.n_tokens<2000]
df_faq

,question,answer,n_tokens
0,How do I get popular services free for 12 months?,"When you create your Azure free account, you s...",107
1,How do I get the 55+ other services that are a...,You start getting monthly free amounts of more...,87
2,What can I do with my $200 credit?,You can apply your credit to try services that...,28
3,Who is eligible for the Azure free account?,The Azure free account is available to all new...,27
4,Do I pay anything to start with the Azure free...,No. Starting is free. You won't be charged any...,27
5,How do I ensure that I won’t be charged?,We’ll never charge you unless you decide to mo...,62
6,Do I have to pay something after 30 days or af...,At the end of your first 30 days or after you ...,104
7,Can I skip the 30 days and just start with pay...,Yes. You can sign up directly for pay as you g...,68
8,What happens once I use my $200 free credit or...,We’ll notify you so you can decide if you want...,82
9,What does it mean to upgrade my account to pay...,"When you sign up for an Azure free account, yo...",111


In [44]:
# understanding tokenization
# https://platform.openai.com/tokenizer

print(df_faq.answer[0])

understand_tokenization = tokenizer.tokenize(df_faq.answer[0])
understand_tokenization
len(understand_tokenization)

When you create your Azure free account, you start getting monthly free amounts of certain types of services. If you move to pay-as-you-go pricing within 30 days or after you’ve used your credit, you’ll continue to receive monthly free amounts of popular services until 12 months after you created your account. After that, if you’re using any of these services, they’ll continue to run and you’ll be billed for them at pay-as-you-go rates.


107

In [15]:
df_faq['curie_search'] = df_faq["answer"].apply(lambda x : get_embedding(x, engine = 'embedding-model'))
df_faq

,question,answer,curie_search
0,How do I get popular services free for 12 months?,"When you create your Azure free account, you s...","[0.0016845327336341143, -0.0040599871426820755..."
1,How do I get the 55+ other services that are a...,You start getting monthly free amounts of more...,"[-0.00166563899256289, -0.011031953617930412, ..."
2,What can I do with my $200 credit?,You can apply your credit to try services that...,"[0.007422318682074547, -0.019402362406253815, ..."
3,Who is eligible for the Azure free account?,The Azure free account is available to all new...,"[0.001902532996609807, -0.023899586871266365, ..."
4,Do I pay anything to start with the Azure free...,No. Starting is free. You won't be charged any...,"[-0.0090416818857193, -0.016852786764502525, 0..."
5,How do I ensure that I won’t be charged?,We’ll never charge you unless you decide to mo...,"[-0.004391007125377655, -0.008358892984688282,..."
6,Do I have to pay something after 30 days or af...,At the end of your first 30 days or after you ...,"[-0.001321667805314064, -0.015075654722750187,..."
7,Can I skip the 30 days and just start with pay...,Yes. You can sign up directly for pay as you g...,"[0.013179963454604149, -0.02787366509437561, 0..."
8,What happens once I use my $200 free credit or...,We’ll notify you so you can decide if you want...,"[-0.00468563474714756, -0.0027697794139385223,..."
9,What does it mean to upgrade my account to pay...,"When you sign up for an Azure free account, yo...","[-0.014927088283002377, -0.018852489069104195,..."


In [17]:
# search through the reviews for a specific product
def search_docs(df, user_query, top_n=3, to_print=True):
    # get embedding for user query
    embedding = get_embedding(
        user_query,
        engine="embedding-model"
    )
    # calculate cosine similarity between user query and all reviews
    df["similarities"] = df.curie_search.apply(lambda x: cosine_similarity(x, embedding))
    # sort by similarity and return top n
    res = (
        df.sort_values("similarities", ascending=False)
        .head(top_n)
    )
    if to_print:
        display(res)
    return res



In [19]:
user_question = "how much the service cost?"
res = search_docs(df_faq, user_question, top_n=2)

,question,answer,curie_search,similarities
9,What does it mean to upgrade my account to pay...,"When you sign up for an Azure free account, yo...","[-0.014927088283002377, -0.018852489069104195,...",0.820252
13,What happens at the end of the 12 months?,"Once you move to pay-as-you-go pricing, certai...","[0.008508202619850636, -0.010522727854549885, ...",0.815805


In [20]:
print(f"best match answer: \n {res.answer.iloc[0]}")
print(f"answer 2: \n {res.answer.iloc[1]}")


best match answer: 
 When you sign up for an Azure free account, you get $200 credit. In the first 30 days, any services you use beyond their free amounts will be deducted from that $200 credit. When you've used up your $200 credit or 30 days have passed (whichever happens first), you'll need to upgrade by moving to pay-as-you-go pricing. That way, you can keep getting free amounts of services and purchase services beyond their free amounts as needed. The cost of those services is charged to the payment method you provide.
answer 2: 
 Once you move to pay-as-you-go pricing, certain types of popular services are available in free monthly amounts for 12 months. If you're using any of these services after 12 months, they'll continue to run and you'll be billed at pay-as-you-go rates. You'll continue to receive monthly free amounts of more than 55 services that are always free.


In [28]:
prompt = f"The user asked: {user_question}, and the answers are:\n\n"
for idx, r in enumerate(res["answer"].values):
    prompt += f"""answer {idx+1}: {r}\n\n"""
prompt += "\n combine the information in "
for i in range(1, len(res["answer"].values)+1):
    prompt += f" answer {i} and "
# remove the last and
prompt = prompt[:-4]
prompt += " and rephrase into a single answer."    
print("The prompt is:\n")
print(prompt)


The prompt is:

The user asked: how much the service cost?, and the answers are:

answer 1: When you sign up for an Azure free account, you get $200 credit. In the first 30 days, any services you use beyond their free amounts will be deducted from that $200 credit. When you've used up your $200 credit or 30 days have passed (whichever happens first), you'll need to upgrade by moving to pay-as-you-go pricing. That way, you can keep getting free amounts of services and purchase services beyond their free amounts as needed. The cost of those services is charged to the payment method you provide.

answer 2: Once you move to pay-as-you-go pricing, certain types of popular services are available in free monthly amounts for 12 months. If you're using any of these services after 12 months, they'll continue to run and you'll be billed at pay-as-you-go rates. You'll continue to receive monthly free amounts of more than 55 services that are always free.


 combine the information in  answer 1 and

In [29]:
response = openai.Completion.create(engine="davinchi-003", prompt=prompt, max_tokens=1024)
text = response['choices'][0]['text'].replace('\n', '').replace(' .', '.').strip()
print(text)

בעת ההרשמה לחשבון חינם של Azure, מקבלים $200 בקרדיט. ב-30 ימים הראשונים, כל שירותים שיושמעלו מחוץ לכמות החינם שלהם יחושבו מהקרדיט הזה של $200. כאשר אתם תמלו את הקרדיט של $200 או תחילו 30 ימים (מהן שהראשונה תתבצע), יהיה עליכם לשדרג בכניסה לתשלום לפי הצרכים. אז אתם יכולים להמשיך לקבל שירותים בחינם ולרכוש שירותים מחוץ לכמות החינם שלהם כפי שנדרש. עלות השירותים הללו תשובע על ידי שיטת התשלום בה תספקו. כאשר אתם תעבירו לתשלום לפי הצרכים, מספר סוגים של שירותים פופולרים יקבלו בחינם לפי חודש למשך 12 חודשים. אם אתם תמשיכו להשתמש באלה שירותים אחרי 12 חודשים, הם ימשיכו לרוץ וישובעו במחירי התשלום לפי הצרכים. תמשיכו לקבל חינמות החודשיות של עוד מעל 55 שירותים שתמיד יהיו חינמים.


In [12]:
prompt = """
How do I get popular services free for 12 months?

"""



response = openai.Completion.create(engine="davinchi-003", prompt=prompt, max_tokens=1024)
text = response['choices'][0]['text'].replace('\n', '').replace(' .', '.').strip()
print(text)


There are several ways to get popular services free for 12 months. First, you may want to explore any special offers or discounts that are available on the service's website. Many companies have promotions that give users free subscription for a certain period of time. Additionally, you can look for coupon codes or promotional links from third parties that provide discounts or free trials for popular services. Finally, be sure to keep an eye out for any special holiday or seasonal promotions that may be available.
